In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


In [2]:
# Tạo dữ liệu mẫu
X = torch.rand(1000, 2)
y = (X[:, 0] + X[:, 1] > 1).float().unsqueeze(1)

dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)



In [3]:
# Định nghĩa mô hình
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.hidden = nn.Linear(2, 4)
        self.output = nn.Linear(4, 1)

    def forward(self, x):
        x = torch.relu(self.hidden(x))
        return self.output(x)  # Không dùng Sigmoid khi dùng BCEWithLogitsLoss

In [4]:
# Hàm huấn luyện
def train_model(loss_fn, optimizer_fn, use_logits_loss=False):
    model = ANN()
    optimizer = optimizer_fn(model.parameters(), lr=0.01)
    loss_function = loss_fn()
    
    for epoch in range(100):
        for batch_X, batch_y in dataloader:
            optimizer.zero_grad()
            output = model(batch_X)
            if use_logits_loss:
                loss = loss_function(output, batch_y)
            else:
                loss = loss_function(torch.sigmoid(output), batch_y)
            loss.backward()
            optimizer.step()
    
    # Tính mất mát và độ chính xác cuối cùng
    with torch.no_grad():
        preds = torch.sigmoid(model(X)) > 0.5
        accuracy = (preds == y).float().mean().item()
        final_loss = loss.item()
    
    return final_loss, accuracy

In [5]:
# Huấn luyện với BCEWithLogitsLoss và Adam
loss_bce_logits, acc_bce_logits = train_model(nn.BCEWithLogitsLoss, optim.Adam, use_logits_loss=True)
# Huấn luyện với BCELoss và SGD
loss_bce_sgd, acc_bce_sgd = train_model(nn.BCELoss, optim.SGD, use_logits_loss=False)


In [6]:
# In kết quả
print(f"BCEWithLogitsLoss + Adam: Loss = {loss_bce_logits:.4f}, Accuracy = {acc_bce_logits:.4f}")
print(f"BCELoss + SGD: Loss = {loss_bce_sgd:.4f}, Accuracy = {acc_bce_sgd:.4f}")

BCEWithLogitsLoss + Adam: Loss = 0.0147, Accuracy = 0.9940
BCELoss + SGD: Loss = 0.3806, Accuracy = 0.9800


## Câu hỏi

### 1. BCEWithLogitsLoss vs BCELoss
- **Mất mát:** BCEWithLogitsLoss có mất mát thấp hơn (0.0147 so với 0.3806), cho thấy nó hoạt động tốt hơn trong bài toán này.
- **Độ chính xác:** BCEWithLogitsLoss đạt độ chính xác 99.4%, trong khi BCELoss với SGD đạt 98.0%.
- **Lý do:** BCEWithLogitsLoss được thiết kế để hoạt động tốt hơn khi đầu ra của mô hình chưa qua Sigmoid, giúp cải thiện độ ổn định số học và tránh tình trạng gradient vanishing.

### 2. Adam vs SGD
- **Tốc độ giảm mất mát:** Adam giảm mất mát nhanh hơn, dẫn đến kết quả cuối cùng tốt hơn.
- **Độ chính xác:** Adam đạt độ chính xác cao hơn so với SGD, do cơ chế điều chỉnh hệ số học tự động giúp tránh bị mắc kẹt ở cực tiểu cục bộ.
- **Lý do:** Adam sử dụng moment và điều chỉnh tốc độ học theo từng tham số, giúp tối ưu hóa hiệu quả hơn so với SGD, đặc biệt khi dữ liệu có độ biến thiên cao.
"""